# Capstone 3: Step 4 - Pre-processing and Training Data Development

# The Data Science Method



1. Problem Identification

2. Data Wrangling

3. Exploratory Data Analysis

4.Pre-processing and Training Data Development


 - Create dummy or indicator features for categorical variables
 - Standardize the magnitude of numeric features
 - Split into testing and training datasets
 - Apply scaler to the testing set




1. Modeling

  - Fit Models with Training Data Set
  - Review Model Outcomes — Iterate over additional models as needed.
  - Identify the Final Model





2. Documentation

   - Review the Results
   - Present and share your findings - storytelling
   - Finalize Code
   - Finalize Documentation



In [23]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import kpss
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA
import warnings
import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
# print the current working directory
print(os.getcwd())

C:\Users\arna_mora\Unit 24


In [24]:
#Loading the datasets

true_df = pd.read_csv('True.csv')
fake_df = pd.read_csv('Fake.csv')

In [25]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [26]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [27]:
true_df['check'] = 'TRUE'
fake_df['check'] = 'FAKE'

In [28]:
true_df.head()

,title,text,subject,date,check
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",TRUE
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",TRUE
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",TRUE
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",TRUE
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",TRUE


In [29]:
fake_df.head()

,title,text,subject,date,check
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",FAKE
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",FAKE
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",FAKE
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",FAKE
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",FAKE


In [53]:
#We will combine both dfs.
df_news = pd.concat([true_df, fake_df])

In [54]:
df_news = pd.concat([true_df, fake_df]).dropna(axis=1)

In [55]:
df_news['check'] = df_news['check'].astype(str)
df_news['check'] = df_news['check'].str.strip()
dict = { 'TRUE' : '1' , 'FAKE' : '0'}
df_news['check'] = df_news['check'].map(dict)
df_news.head()

,title,text,subject,date,check
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


# Extracting the training data

In [10]:
y = df_news["check"] 
X = df_news.drop("check", axis=1)

# Make training and test sets 
X_train, X_test, y_train, y_test = train_test_split(df_news['text'], y, test_size=0.33, random_state=53)

# Building Vectorizer Classifiers

To get a good idea if the words and tokens in the articles had a significant impact on whether the news was fake or real, we should begin by using CountVectorizer and TfidfVectorizer.

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
# Initialize the 'count_vectorizer' 
count_vectorizer = CountVectorizer(stop_words='english')

# Fit and transform the training data 
count_train = count_vectorizer.fit_transform(X_train) 

# Transform the test set 
count_test = count_vectorizer.transform(X_test)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Initialize the 'tfidf_vectorizer' 
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7) 

# Fit and transform the training data 
tfidf_train = tfidf_vectorizer.fit_transform(X_train) 

# Transform the test set 
tfidf_test = tfidf_vectorizer.transform(X_test)

In [16]:
# Get the feature names of `tfidf_vectorizer` 
print(tfidf_vectorizer.get_feature_names()[-10:])

# Get the feature names of `count_vectorizer` 
print(count_vectorizer.get_feature_names()[:10])

['zzjjpdaivn', 'zzn3bqnfsk', 'zzpx_bzka40police', 'zzqvyk8xif', 'zztaine', 'zzuml4hkoc', 'zzzzaaaacccchhh', 'zzzzzzzz', 'zzzzzzzzzzzzz', 'émigré']
['00', '000', '0000', '00000017', '00004', '00007', '000270', '00042', '0005', '0009']
